# Day 13

## Part 1

With your help, the hot springs team locates an appropriate spring which launches you neatly and precisely up to the edge of Lava Island.

There's just one problem: you don't see any lava.

You do see a lot of ash and igneous rock; there are even what look like gray mountains scattered around. After a while, you make your way to a nearby cluster of mountains only to discover that the valley between them is completely full of large mirrors. Most of the mirrors seem to be aligned in a consistent way; perhaps you should head in that direction?

As you move through the valley of mirrors, you find that several of them have fallen from the large metal frames keeping them in place. The mirrors are extremely flat and shiny, and many of the fallen mirrors have lodged into the ash at strange angles. Because the terrain is all one color, it's hard to tell where it's safe to walk or where you're about to run into a mirror.

You note down the patterns of ash (.) and rocks (#) that you see as you walk (your puzzle input); perhaps by carefully analyzing these patterns, you can figure out where the mirrors are!

For example:

```
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#
```
To find the reflection in each pattern, you need to find a perfect reflection across either a horizontal line between two rows or across a vertical line between two columns.

In the first pattern, the reflection is across a vertical line between two columns; arrows on each of the two columns point at the line between the columns:

```
123456789
    ><   
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.
    ><   
123456789
```
In this pattern, the line of reflection is the vertical line between columns 5 and 6. Because the vertical line is not perfectly in the middle of the pattern, part of the pattern (column 1) has nowhere to reflect onto and can be ignored; every other column has a reflected column within the pattern and must match exactly: column 2 matches column 9, column 3 matches 8, 4 matches 7, and 5 matches 6.

The second pattern reflects across a horizontal line instead:

```
1 #...##..# 1
2 #....#..# 2
3 ..##..### 3
4v#####.##.v4
5^#####.##.^5
6 ..##..### 6
7 #....#..# 7
```
This pattern reflects across the horizontal line between rows 4 and 5. Row 1 would reflect with a hypothetical row 8, but since that's not in the pattern, row 1 doesn't need to match anything. The remaining rows match: row 2 matches row 7, row 3 matches row 6, and row 4 matches row 5.

To summarize your pattern notes, add up the number of columns to the left of each vertical line of reflection; to that, also add 100 multiplied by the number of rows above each horizontal line of reflection. In the above example, the first pattern's vertical line has 5 columns to its left and the second pattern's horizontal line has 4 rows above it, a total of 405.

Find the line of reflection in each of the patterns in your notes. What number do you get after summarizing all of your notes?

In [75]:
from IPython.display import Markdown
from typing import Optional
import pandas as pd


def find_mirror_position(df: pd.DataFrame, v: bool = False, show=False) -> Optional[int]:
    max_score = 0
    for i in range(1, len(df) + 1):
        if i == len(df):
            # can't split on the last row
            continue

        top_rows = df.iloc[:i]
        bottom_rows = df.iloc[i:]

        if len(top_rows) > len(bottom_rows):
            # shorten top rows
            top_rows = top_rows.tail(len(bottom_rows))
        elif len(top_rows) < len(bottom_rows):
            # shorten bottom rows
            bottom_rows = bottom_rows.head(len(top_rows))

        # flip the bottom rows' index positions so they (potentially) match the top
        bottom_rows = bottom_rows.iloc[::-1]

        assert len(bottom_rows) == len(top_rows)

        # string comparisons on just the values is better than trying to compare the whole dataframe
        # portions, which will include the index values and columns by default
        t = top_rows.to_string(index=False, header=False)
        b = bottom_rows.to_string(index=False, header=False)
        if t == b:
            df_copy = df.copy()
            if v:
                # debug_df = debug_df.transpose()
                df_copy = df_copy.transpose()

            # last row's index + 1 in `top_rows` is the position of the mirror line
            last_index = top_rows.index[-1] + 1
            # 1x for mirroring against a vertical line, 100x for a horizontal line
            score = last_index * 100 if not v else last_index

            if show:
                display(
                    Markdown(
                        f"### {'vertical' if v else 'horizontal'} line between {last_index-1}-{last_index}: {last_index} total row(s) -> {score=}"
                    )
                )
                print(f"{last_index=}")

                def stuff(s, b: int):
                    return ["border: 3px dashed gold; color:gold" if s.name <= b else "" for v in s]

                display(
                    df_copy.style.apply(
                        stuff,
                        b=last_index,
                        axis=int(not v),
                    )
                    .map_index(
                        lambda x: "background-color: #99000077"
                        if x in top_rows.index
                        else "background-color: #00009977"
                        if x in bottom_rows.index
                        else "",
                        axis=int(v),
                    )
                    .map(lambda x: "background-color: #00cccc77" if x == 1 else "")
                    .map_index(
                        lambda x: "color:gold" if x <= max(top_rows.index) else None,
                        axis=int(v),
                    )
                )

            max_score += score
    return max_score


def find_mirror_lines(data: str, show: bool = False):
    rows = data.split("\n")
    df = pd.DataFrame([list(line) for line in rows]).astype(int)

    score = 0

    horizontal_line_score = find_mirror_position(df, show=show)
    if horizontal_line_score is not None:
        score += horizontal_line_score

    vertical_line_score = find_mirror_position(df.transpose(), v=True, show=show)
    if vertical_line_score is not None:
        score += vertical_line_score

    return score


test_input = (
    """
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#""".strip()
    .replace("#", "1")
    .replace(".", "0")
    .split("\n\n")
)

test_total_score = 0
for i, test_block in enumerate(test_input):
    display(Markdown(f"# `{i}`"))
    test_score = find_mirror_lines(test_block)
    test_total_score += test_score
print(test_total_score)
assert test_total_score == 405

# `0`

# `1`

405


In [76]:
inputs = open("../inputs/13.txt").read().replace("#", "1").replace(".", "0").split("\n\n")

focus_nums = [19, 97]

score = 0
for i, block in enumerate(inputs):
    if i in focus_nums:
        display(Markdown(f"# `{i}`"))
    score += find_mirror_lines(block, show=i in focus_nums)

# `19`

### vertical line between 12-13: 13 total row(s) -> score=13

last_index=13


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,1,0,1,1,0,0,0,0,0,1,0,1,1,0,1,0
1,0,1,1,0,0,0,1,1,1,1,0,1,1,1,1,0,1
2,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0
3,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1
4,0,1,0,0,0,1,0,1,0,1,1,0,0,0,0,1,1
5,1,0,1,0,1,0,0,1,1,1,0,0,0,0,0,0,1
6,0,1,0,1,1,0,0,0,0,0,1,1,1,1,1,1,0
7,0,1,0,1,1,0,0,0,0,0,1,1,1,1,1,1,0
8,1,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,1


# `97`

### vertical line between 9-10: 10 total row(s) -> score=10

last_index=10


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,1,1,1,1,1,0,0,1,1,1,1
3,1,1,1,0,1,0,0,1,0,1,1,0,1,0,0
4,0,1,0,0,0,0,1,1,0,1,1,0,1,1,0
5,1,1,1,0,0,0,1,1,0,1,1,0,1,1,0
6,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0


In [77]:
score

27664

## Part 2

You resume walking through the valley of mirrors and - SMACK! - run directly into one. Hopefully nobody was watching, because that must have been pretty embarrassing.

Upon closer inspection, you discover that every mirror has exactly one smudge: exactly one . or # should be the opposite type.

In each pattern, you'll need to locate and fix the smudge that causes a different reflection line to be valid. (The old reflection line won't necessarily continue being valid after the smudge is fixed.)

Here's the above example again:

```
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#
```
The first pattern's smudge is in the top-left corner. If the top-left # were instead ., it would have a different, horizontal line of reflection:

```
1 ..##..##. 1
2 ..#.##.#. 2
3v##......#v3
4^##......#^4
5 ..#.##.#. 5
6 ..##..##. 6
7 #.#.##.#. 7
```
With the smudge in the top-left corner repaired, a new horizontal line of reflection between rows 3 and 4 now exists. Row 7 has no corresponding reflected row and can be ignored, but every other row matches exactly: row 1 matches row 6, row 2 matches row 5, and row 3 matches row 4.

In the second pattern, the smudge can be fixed by changing the fifth symbol on row 2 from . to #:

```
1v#...##..#v1
2^#...##..#^2
3 ..##..### 3
4 #####.##. 4
5 #####.##. 5
6 ..##..### 6
7 #....#..# 7
```
Now, the pattern has a different horizontal line of reflection between rows 1 and 2.

Summarize your notes as before, but instead use the new different reflection lines. In this example, the first pattern's new horizontal line has 3 rows above it and the second pattern's new horizontal line has 1 row above it, summarizing to the value 400.

In each pattern, fix the smudge and find the different line of reflection. What number do you get after summarizing the new reflection line in each pattern in your notes?

In [156]:
def find_mirror_position2(df: pd.DataFrame, v: bool = False) -> Optional[int]:
    output = {
        "max_score": 0,
        "vertical": v,
        "position": None,
    }

    for i in range(1, len(df) + 1):
        if i == len(df):
            # can't split on the last row
            continue

        top_rows = df.iloc[:i]
        bottom_rows = df.iloc[i:]

        if len(top_rows) > len(bottom_rows):
            # shorten top rows
            top_rows = top_rows.tail(len(bottom_rows))
        elif len(top_rows) < len(bottom_rows):
            # shorten bottom rows
            bottom_rows = bottom_rows.head(len(top_rows))

        # flip the bottom rows' index positions so they (potentially) match the top
        bottom_rows = bottom_rows.iloc[::-1]

        assert len(bottom_rows) == len(top_rows)

        # string comparisons on just the values is better than trying to compare the whole dataframe
        # portions, which will include the index values and columns by default
        t = top_rows.to_string(index=False, header=False)
        b = bottom_rows.to_string(index=False, header=False)
        if t == b:
            # last row's index + 1 in `top_rows` is the position of the mirror line
            last_index = top_rows.index[-1] + 1
            output["position"] = last_index
            # 1x for mirroring against a vertical line, 100x for a horizontal line
            score = last_index * 100 if not v else last_index
            output["max_score"] = score

    return output


def find_mirror_lines2(data: str):
    rows = data.split("\n")
    df = pd.DataFrame([list(line) for line in rows]).astype(int)

    results = {"max_score": 0}

    horizontal_results = find_mirror_position2(df)
    if horizontal_results["max_score"] > results["max_score"]:
        results = horizontal_results

    vertical_results = find_mirror_position2(df.transpose(), v=True)
    if vertical_results["max_score"] > results["max_score"]:
        results = vertical_results

    return results


def flip_smudge_and_score(input_str: str) -> int:
    input_data: list[list[str]] = [list(line) for line in input_str.split("\n")]

    orig_mirror = find_mirror_lines2(input_str)
    print(f"{orig_mirror=}")

    score = 0
    for row_num, row in enumerate(input_data):
        for col_num, char in enumerate(row):
            # get ready to flip one character
            new_char = "0" if char == "1" else "1"
            flip_str = f"{row_num},{col_num} flip from `{'#' if char == '1' else '.'}` to `{'#' if new_char == '1' else '.'}`"

            # flip from `#` to `.` (or vice-versa)
            input_data[row_num][col_num] = new_char

            # convert back to a string and check for (new) mirrors
            smudge_replaced_str = "\n".join("".join(row) for row in input_data)
            new_mirror = find_mirror_lines2(smudge_replaced_str)

            # flip back so we don't mess with the next iteration
            input_data[row_num][col_num] = char

            if new_mirror.get("max_score"):
                # check if it's different from the original
                new = new_mirror != orig_mirror
                # print(f"{new_mirror=}: {flip_str} {new=} {'*' * 3 if new else ''}")
                if new:
                    print(
                        f"\t{flip_str} makes a {'vertical' if new_mirror['vertical'] else 'horizontal'} line with score {new_mirror['max_score']} (pos@{new_mirror['position']}) {new=}"
                    )
                    score = new_mirror["max_score"]
    return score


test_input2 = (
    """#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#""".strip()
    .replace("#", "1")
    .replace(".", "0")
    .split("\n\n")
)

test_total_score = 0
for i, test_block in enumerate(test_input2):
    display(Markdown(f"# `{i}`"))
    test_score = flip_smudge_and_score(test_block)
    test_total_score += test_score
    display(Markdown(f"#### `{test_score=}`"))
display(Markdown(f"# `{test_total_score=}`"))
assert test_total_score == 400

# additional sample input to test
# https://www.reddit.com/r/adventofcode/comments/18hitog/2023_day_13_easy_additional_examples/
test_input2a = (
    """#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#

.#.##.#.#
.##..##..
.#.##.#..
#......##
#......##
.#.##.#..
.##..##.#

#..#....#
###..##..
.##.#####
.##.#####
###..##..
#..#....#
#..##...#

#.##..##.
..#.##.#.
##..#...#
##...#..#
..#.##.#.
..##..##.
#.#.##.#.""".strip()
    .replace("#", "1")
    .replace(".", "0")
    .split("\n\n")
)

test_total_score = 0
for i, test_block in enumerate(test_input2a):
    display(Markdown(f"# `{i}`"))
    test_score = flip_smudge_and_score(test_block)
    test_total_score += test_score
    display(Markdown(f"#### `{test_score=}`"))
display(Markdown(f"# `{test_total_score=}`"))
assert test_total_score == 1400

# `0`

orig_mirror={'max_score': 5, 'vertical': True, 'position': 5}
	0,0 flip from `#` to `.` makes a horizontal line with score 300 (pos@3) new=True
	5,0 flip from `.` to `#` makes a horizontal line with score 300 (pos@3) new=True


#### `test_score=300`

# `1`

orig_mirror={'max_score': 400, 'vertical': False, 'position': 4}
	1,4 flip from `.` to `#` makes a horizontal line with score 100 (pos@1) new=True


...

#### `test_score=600`

# `4`

orig_mirror={'max_score': 0}


#### `test_score=0`

# `test_total_score=1400`

In [157]:
score2 = 0
for i, block in enumerate(inputs):
    display(Markdown(f"# `{i}`"))
    new_score = flip_smudge_and_score(block)
    display(Markdown(f"#### `{new_score=}`"))
    score2 += new_score
    break
score2

# `0`

orig_mirror={'max_score': 12, 'vertical': True, 'position': 12}


#### `new_score=0`

0

In [134]:
score2

24891

- 31976 too low